In [18]:
%cd /content/drive/MyDrive/
# raw_data is imported from global config
root_dir='/content/drive/MyDrive/'  # ChangeThis
result_dir=root_dir+'grasp/TSception/resultBandInput/'

/content/drive/MyDrive


In [19]:
! pip install mne==0.19.2;
! pip install torch;

In [20]:
import os
os.getcwd()

'/content/drive/MyDrive'

In [21]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

from grasp.utils import SEEGDataset
from grasp.TSception.utils import regulization
from grasp.TSception.Models import TSception
# load the data
from grasp.utils import rawData

In [23]:
import sys, importlib
importlib.reload(sys.modules['grasp.TSceptionWithoutMovement2.Models'])
from grasp.TSception.Models import TSception2


KeyError: ignored

In [ ]:
import inspect as i
import sys
#sys.stdout.write(i.getsource(rawData2));

In [ ]:
enable_cuda = torch.cuda.is_available()
print('GPU computing: ', enable_cuda)

In [ ]:
%%capture
# suppress the output
import sys, importlib
importlib.reload(sys.modules['grasp.utils'])
from grasp.utils import rawData2
sampling_rate=1000
traindata, valdata, testdata = rawData2(split=True,move2=False);  # (20, 15000, 118) (channels, time, trials)
traindata = traindata.transpose(2, 0, 1)  # (118, 20, 15000) (trials,channels,  time)
valdata = valdata.transpose(2, 0, 1) # (8, 20, 15000)
testdata = testdata.transpose(2, 0, 1)  # (8, 20, 15000)
trainx, trainy = traindata[:, :-1, :], traindata[:, -1, :]+0.05
valx, valy = valdata[:, :-1, :], valdata[:, -1, :]+0.05
testx, testy = testdata[:, -1, :], testdata[:, -1, :]+0.05

dataset_train = SEEGDataset(trainx, trainy)
dataset_val = SEEGDataset(valx, valy)
dataset_test = SEEGDataset(testx, testy)

In [ ]:
# Dataloader for training process
train_loader = DataLoader(dataset=dataset_train, batch_size=1, shuffle=True, pin_memory=False)
val_loader = DataLoader(dataset=dataset_val, batch_size=1, pin_memory=False)
test_loader = DataLoader(dataset=dataset_test, batch_size=1, pin_memory=False)

chnNum=19
learning_rate=0.001
epochs=500
step=50 #ms
T=1000 #ms
totalLen=15000 #ms
batch_size=int((totalLen-T)/step) # 280
num_T = 3 # (6 conv2d layers) * ( 3 kernel each layer)
num_S = 3
hidden_size=222
dropout=0.2
Lambda = 1e-6

# __init__(self,input_size, sampling_rate, num_T, num_S, hiden, dropout_rate)
#net = TSception2(chnNum,sampling_rate, num_T, num_S, batch_size).float()
net = IMVTensorLSTM(114, 1, 500)
if(enable_cuda):
	net.cuda()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
criterion = nn.MSELoss()


for epoch in range(epochs):
    print("------ epoch " + str(epoch) + " -----")
    net.train()

    loss_epoch=0
    #trial=0
    for trial, (trainx, trainy) in enumerate(train_loader): # ([1, 15000, 19]), ([1, 15000])
        # debug on first trail
        if trial == 1:
            #break
            pass
        optimizer.zero_grad()

        x = np.zeros((batch_size, 1, chnNum, T)) # 4D:(280,1,19,1000ms):(batch_size, planes, height, weight)
        target = np.zeros((batch_size,1)) # (280, 1)

        # format 1 trial into 3D tensor
        for bs in range(batch_size):
            x[bs, 0, :, :] = trainx[0, :, bs*step:(bs*step + T)]
            target[bs,0] = trainy[0,bs*step + T +1]
        if (enable_cuda):
            x= torch.from_numpy(x).float().cuda()
            target = torch.from_numpy(target).cuda()
        #y_pred = net(x)
        # try the IMV model
        y_pred = net(torch.from_numpy(x.squeeze().transpose(0, 2, 1)).float())
        y_pred = np.expand_dims(y_pred, axis=1)

        # regularization
        loss1 = criterion(y_pred, target.float())
        loss2 = regulization(net, Lambda)
        loss=loss1+loss2
        loss.backward()
        optimizer.step()

        ls=loss1.item()
        loss_epoch+=ls
        #print("Loss: " + str(ls) + ".")
        #trial+=1
    print("Loss:"+str(loss_epoch/(trial+1))+".")
    if epoch % 1 ==0:
        net.eval()
        print("Validating.")
        with torch.no_grad():
            vpredAll = []
            vtargetAll = []
            for trial, (valx, valy) in enumerate(val_loader):  # ([1, 15000, 19]), ([1, 15000])
                #print("Validating on trial " + str(trial) + ".")

                vx = np.zeros((batch_size, 1, chnNum, T))  # 4D:(?,1,19,1000ms):(batch_size, planes, height, weight)
                vtarget = np.zeros((batch_size, 1))

                # format 1 trial into 3D tensor
                for bs in range(batch_size):
                    vx[bs, 0, :, :] = valx[0, :, (bs * step):(bs * step + T)]
                    vtarget[bs, 0] = valy[0,bs * step + T + 1]
                if (enable_cuda):
                    vx= torch.from_numpy(vx).float().cuda()
                    vtarget = torch.from_numpy(vtarget).cuda()
                #y_pred = net(vx)
                # try IMV model
                y_pred = net(torch.from_numpy(vx.squeeze().transpose(0,2,1)).float())
                y_pred=np.expand_dims(y_pred, axis=1)
                
                vpredAll.append(y_pred.cpu().data.numpy())
                vtargetAll.append(vtarget.cpu().data.numpy())
        vpredAll = np.concatenate(vpredAll,axis=0)
        vtargetAll = np.concatenate(vtargetAll, axis=0)

        fig, ax = plt.subplots(figsize=(6, 3))
        plt.ion()
        ax.clear()
        ax.plot(vtargetAll, label="True", linewidth=1)
        ax.plot(vpredAll, label='Predicted - Test', linewidth=1)
        ax.legend(loc='upper left')
        figname = result_dir+'prediction' + str(epoch) + '.png'
        fig.savefig(figname)
        plt.close(fig)
